# Hill Cipher
This code implements the Hill Cipher, in order to encrypt or decrypt a message.

In [92]:
%pip install Unidecode

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In order to remove national characters, we are going to use unidecode. 
codecs library allows to read a file that contains polish letters

In [93]:
from unidecode import unidecode
#unidecode('Naprawdę lubię kryptografię')
import codecs

Since the algorithm uses matrices, we need numpy to make it work

In [94]:
import numpy as np
import math

This function removes any character that is not a letter or a number, such as spaces, enters, whitespace characters, etc.

In [95]:
def removeUnnecesaryChars(text):
    return ''.join(letter for letter in text if letter.isalpha())
#removeUnnecesaryChars('Naprawde lubie kryptografie')

This function calculates the inverse of a number A modulo M.

In [132]:
def modInverse(A, M):
 
    for X in range(1, M):
        if ((A* X) % M == 1):
            return X
    return -1

This function calculates the inverse of a matrix of dimension 3x3. First, it calculates the inverse of the determinant using the function above. 
Then we calculate the matrix inverse using the formula in the book.

In [97]:
def inverseMatrixDim3(matrix):
    detMatrix = int(np.linalg.det(matrix))
    inverseDetMatrix = modInverse(detMatrix%26,26)

    a = matrix[0,0]
    b = matrix[0,1]
    c = matrix[0,2]
    d = matrix[1,0]
    e = matrix[1,1]
    f = matrix[1,2]
    g = matrix[2,0]
    h = matrix[2,1]
    i = matrix[2,2]

    inverse = np.array([[e*i-f*h,c*h-b*i,b*f-c*e],
                        [f*g-d*i,a*i-c*g,c*d-a*f],
                        [d*h-e*g,b*g-a*h,a*e-b*d]])
    
    for r in range(3):
        for c in range(3):
            inverse[r,c] = ((inverse[r,c]%26)*inverseDetMatrix)%26
    return inverse

#inverseMatrixDim3(np.array([[ 6, 24,  1],[13, 16, 10],[20, 17, 15]]))

25


array([[ 8,  5, 10],
       [21,  8, 21],
       [21, 12,  8]])

This function calculates the inverse of a matrix of dimension 2x2. It also uses the formula in the book.

In [98]:
def inverseMatrixDim2(matrix):
    inverse = np.array([[0,0],[0,0]])
    detMatrix = int(np.linalg.det(matrix))
    inverseDetMatrix = modInverse(detMatrix,26)
    inverse[0,0] = (matrix[1,1]*inverseDetMatrix)%26
    inverse[0,1] = (-matrix[0,1]*inverseDetMatrix)%26
    inverse[1,0] = (-matrix[1,0]*inverseDetMatrix)%26
    inverse[1,1] = (matrix[0,0]*inverseDetMatrix)%26
    return inverse
    
# inverseMatrixDim2(np.array([[19,  0, 13],[ 6,  4, 17],[ 8, 13,  4]])) 

array([[22,  0],
       [ 6,  7]])

This function calculates a matrix of numbers given a string of letters. 
It receives dim1 as an argument, which is the number of columns in the resulting matrix. 
The number of rows depends on the number of columns and the length of the string. If there are some positions in the last row left, we fill them with random numbers between 0 and 25.

So we create a matrix text_matrix with size (dim0,dim1) and fil it with random numbers modulo 26. Then convert each character of the text to a number and we fill the matrix using those numbers.

In [99]:
def stringToMatrix(text,dim1):
    text = text.lower()
    dim0 = int(len(text)/dim1 + len(text)%dim1)
    text_matrix = np.random.randint(0,25,(dim0, dim1))
    row = 0
    column = 0
    for character in text:
        number = (ord(character) - ord('a'))%26
        text_matrix[row,column] = number
        if(column == dim1-1):
            row += 1
        column = (column+1)%dim1
    return text_matrix

# stringToMatrix("GYBNQKURP",3)


array([[ 6, 24,  1],
       [13, 16, 10],
       [20, 17, 15]])

the key is correct if the key can be converted to a matrix nxn. Also, we need the matrix of the key be invertible. We know that a matrix is invertible modulo 26 if and only if gcd(determinant,26)=1.

In [100]:
def isCorrectKey(key,dim):
    if len(key) != dim*dim:
        return False
    key_matrix = stringToMatrix(key,dim)
    determinant = np.linalg.det(key_matrix)
    if(math.gcd(int(determinant),26) != 1):
        return False
    return True


For encrypting the text with the key:

1.- Separate each row of the text converted to matrix

2.- Make the product of the key matrix and the row modulo 26

3.- Finally convert the numbers of the product to letters

In [112]:
def encryptText(text, key, dim):
   encrypted_text = ""
   for row in stringToMatrix(text, dim):
      block = (np.matmul(stringToMatrix(key,dim),row))%26
      for number in block:
         encrypted_text += chr(ord('a') + int(number))
   return encrypted_text

# encryptText("act","GYBNQKURP",3)

'poh'

For decrypting the text with the key:

1.- Calculate the inverse matrix of the key, depending on the dimension

2.- Separate the text matrix in rows 

3.- Calculate the product of the inverse matrix and the row modulo 26

4.- Convert the final product to letters to get the decrypted text

In [102]:
def decryptText(text, key, dim):
    decrypted_text = ""
    if dim==2:
        inverse_key_matrix = inverseMatrixDim2(stringToMatrix(key,2))
    elif dim == 3:
        inverse_key_matrix = inverseMatrixDim3(stringToMatrix(key,3))
    else:
        return "This dimension is not available yet"

    for row in stringToMatrix(text,dim):
        block = np.matmul(inverse_key_matrix,row)
        for number in block:
            decrypted_text += chr(ord('a') + int(number)%26)
    return decrypted_text

# decryptText("poh","GYBNQKURP",3)

25
[[ 8  5 10]
 [21  8 21]
 [21 12  8]]


'act'

We read the text and modify it, if necessary, so that it is in a valid format. That is, removing unnecessary characters (including numbers) and substituting national characters.

In [128]:
file = codecs.open('text.txt','r','utf-8')
text = file.read()
file.close()

text = text.lower()
text = unidecode(text)
text = removeUnnecesaryChars(text)

Reading the key from key.txt and checking that the key is valid

In [129]:
file = codecs.open('key.txt','r','utf-8')
key = file.read()
file.close()

key = key.lower()
correct_key = isCorrectKey(key,2)

Now we ask if the user wants to encrypt or decrypt the message and save the result in the corresponding file.

In [131]:
encrypt_option = input('Press 1 for encrypting a message and press 0 for decrypting a message')
if encrypt_option == '1' and correct_key:
    file = open('text_en.txt', 'w')
    file.write(str(encryptText(text,key,2)))

elif encrypt_option == '0' and correct_key:
    file = open('text_de.txt', 'w')
    file.write(str(decryptText(text, key,2)))

else:
    print('The values introduced are not valid')

file.close()